## 데이터셋 EDA

| 구분        | 개수  | fake여부   | 기반   | 사이즈     | train(%) | val(%) | test(%) |
| ----------- | ----- | ---------- | ------ | ---------- | -------- | ------ | ------- |
| FFHQ        | 35000 | 0          |        | 1024, 1024 | 70       | 10     | 20      |
| LFW         | 13233 | 0          |        | 250, 250   | 70       | 10     | 20      |
| CelebA      | 17767 | 0          |        | 178, 218   | 70       | 10     | 20      |
| StarGAN     | 6000  | 1          | CelebA | 1024, 1024 | 70       | 10     | 20      |
| StyleGAN1   | 30000 | 1          | FFHQ   | 1024, 1024 | 70       | 10     | 20      |
| StyleGAN2   | 30000 | 1          | FFHQ   | 1024, 1024 | 70       | 10     | 20      |
| StyleGAN-XL | 13200 | 1 (unseen) | FFHQ   | 256, 256   | 0        | 0      | 100     |



각 데이터는  아래와 같은 형태로 저장되어 있다.

```
dataset
├── CelebA
├── FFHQ
├── LFW
├── StarGAN
├── StyleGAN1
├── StyleGAN2
└── StyleGANXL
```
real, fake 얼굴 사진 각각 66,000 장을 사용한다. StyleGANXL 데이터는 학습에 사용하지 않고, 테스트 시에만 사용한다.

학습 과정에서 모든 데이터를 램에 올리기에는 데이터의 양이 너무 많기 때문에 train, val, test 데이터 들의 경로가 포함된  csv 파일을 만들어준다.



# 1. 데이터셋 준비

In [5]:
import pathlib
import os
import pandas as pd
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

In [6]:
def makeCSV(path, label, tag, dst=None):
    """
    path: 사진이 들어있는 폴더의 경로
    label: 0이면 진짜 사진, 1이면 가짜 사진
    tag: 이후에 분석을 쉽게 하기 위해 폴더 이름을 tag로 넣어줌

    return: path 내부에 있는 모든 사진의 경로와 label, tag를 행으로 갖는 csv 파일
    """
    path = pathlib.PosixPath(path) # path 설정
    generator = path.glob("./*") # 경로 이하의 파일을 뱉는 generator 생성
    nullList = list() # 빈 파일 경로가 들어있는 리스트
    fileList = list() # 정상적인 파일의 경로가 들어있는 리스트

    i = 0
    while True:
        i += 1
        try:
            filepath = generator.__next__() # 파일의 경로를 하나씩 받아온다
            if os.stat(filepath).st_size == 0: # 파일 사이즈가 0인 null file을 검사한다
                nullList.append(filepath)
            else:
                fileList.append(filepath)
        except: # 더 이상 남은 파일이 없으면 while 문을 빠져나간다
            break
        if i % 1000 == 0: # 1000 간격으로 진행도를 보여준다
            clear_output()
            print(i)
    csv = pd.DataFrame({
        'filepath': fileList,
        'label': label,
        'tag': tag
    })

    if dst: # 저장할 경로가 존재하면 저장하고 리턴값은 None
        csv.to_csv(dst)
        return
    
    return csv, nullList

In [9]:
dirList = list(pathlib.PosixPath("./dataset").glob("./*"))

# 아래 값을 적절하게 조절해서 csv 폴더 생성, dirList에는 dataset 폴더들의 이름이 들어 있음
makeCSV(dirList[0], label=0, tag="FFHQ", dst=f"./csv/{dirList[0].name}")

In [ ]:
csvList = list(pathlib.PosixPath("./csv/").glob("./*.csv"))
# 여기에 styleGAN-XL에 해당하는 index만 pop으로 솎아준다
csvList.pop(4)

# XL을 제외한 모든 데이터들을 넣어준다
df = pd.DataFrame()
for csv in csvList:
    temp = pd.read_csv(csv, index_col=0)
    df = pd.concat([df, temp], axis=0)

In [ ]:
# 지금은 안 섞인 상태니까 한번 섞어준다
train, test = train_test_split(df ,test_size=0.2, shuffle=True)

train, val = train_test_split(train, test_size=0.125)

train.to_csv("./csv/train.csv")
val.to_csv("./csv/val.csv")
test.to_csv("./csv/test.csv")

In [ ]:
# XL 전용 테스트 
xl = pd.read_csv("./csv/StyleGANXL", index_col=0)
xl = xl.sample(13200)
test = test[test.label == 0]

test_xl = pd.concat([xl, test], axis=0)
# 한번 섞어서 저장한다
test_xl = test_xl.sample(len(test_xl))
test_xl.to_csv("./csv/test_XL.csv")

## 1 - 1. 데이터 전처리 - Benford First Digits Probability Distribution

각 이미지를 DCT 를 이용해 전처리하고 한 이미지에서 각 digit의 첫 번쨰 숫자 빈도를 센다

In [ ]:
import cv2 
import numpy as np

def compute_first_digits(img, normalize=True):
    # 이미지를 gray scale로 읽어옴
    if isinstance(img, str):
        img = cv2.imread(img, 0)
    # 컬러 이미지가 들어오면 gray scale 로 바꿔줌
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    

    # dct 는 shape이 짝수여야 한다고 함, 홀수면 처리를 해줌
    if img.shape[0] % 2:
        img = img[:img.shape[0] - 1,:]
    
    if img.shape[1] % 2:
        img = img[:,:img.shape[1] - 1]

    # 0 ~ 255 로 normalize 
    if normalize:
        img = cv2.normalize(img, np.zeros(img.shape), 0, 255, cv2.NORM_MINMAX)
    
    # 0 ~ 1 로 다시 normalize 후 descrete cosine transform, 어짜피 First Digit 만 구할꺼니 abs 까지
    img = cv2.dct(np.float32(img) / 255.0)
    img = np.abs(img)

    # 0 인 경우 로그에 들어갈 때 문제가 생김
    img = img[img != 0]
    min_val = img.min()

    if min_val < 1:
        img = np.power(10, -np.floor(np.log10(min_val))) * img

    # 위 코드를 지나면 모두 1.0 보다 커야함
    if not (img >= 1.0).all():
        raise ValueError("Error")
    
    # [1, 10) 은 0, [10, 100) 은 1, 이런느낌
    digits = np.log10(img).astype(int).astype('float32')
    first_digits = img / np.power(10, digits)
    first_digits = first_digits.astype(int)

    # 혹시라도 0이 들어있는 부분이 있다면 예외처리해준다
    first_digits = first_digits[first_digits != 0]

    
    return first_digits

def compute_first_digits_counts(img, normalise=True):
    first_digits = compute_first_digits(img, normalise)
    unq, counts = np.unique(first_digits, return_counts=True)
    return unq, counts

def compute_first_digits_prob(img, normalise=True):
    unq, counts = compute_first_digits_counts(img, normalise)
    counts = counts / sum(counts)
    return counts

In [ ]:
# train val test XL 각각의 파일경로들을 불러온다
train_filepath = pd.read_csv("./csv/train.csv", index_col=0).filepath.values
val_filepath = pd.read_csv("./csv/val.csv", index_col=0).filepath.values
test_filepath = pd.read_csv("./csv/test.csv", index_col=0).filepath.values
test_XL_filepath = pd.read_csv("./csv/test_XL.csv", index_col=0).filepath.values

In [ ]:
# 불러온 경로들을 사용해 위에서 미리 정의한 함수들을 사용해서 확률분포로 전처리해줌
train = np.array([compute_first_digits_prob(i) for i in train_filepath])
val = np.array([compute_first_digits_prob(i) for i in val_filepath])
test = np.array([compute_first_digits_prob(i) for i in test_filepath])
test_XL = np.array([compute_first_digits_prob(i) for i in test_XL_filepath])

In [ ]:
# npy 폴더에 저장
np.save("./npy/train", train)
np.save("./npy/val", val)
np.save("./npy/test", test)
np.save("./npy/test_XL", test_XL)

In [ ]:
# 전처리가 완료된 확률분포를 불러옴
x_train = np.load("./npy/train")
x_val = np.load("./npy/val")
x_test = np.load("./npy/test")
x_test_XL = np.load("./npy/test_XL")

In [ ]:
# label도 따로 불러옴
y_train = pd.read_csv("./csv/train.csv", index_col=0).label.values
y_val = pd.read_csv("./csv/val.csv", index_col=0).label.values
y_test = pd.read_csv("./csv/test.csv", index_col=0).label.values
y_test_XL = pd.read_csv("./csv/test_XL.csv", index_col=0).label.values

train, val 을 합쳐준다

In [ ]:
x_train = np.concatenate([x_train, x_val], axis=0)
y_train = np.concatenate([y_train, y_val], axis=0)
del x_val, y_val

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix as cm, classification_report as cr

전처리한 결과를 이용해 머신러닝 시작

In [ ]:
rf = RandomForestClassifier(verbose=1, n_jobs=-1)
rf.fit(x_train, y_train)
y_rf_pred = rf.predict(x_test)
y_rf_proba = rf.predict_proba(x_test)

print(cm(y_true=y_test, y_pred=y_rf_pred))
print(cr(y_true=y_test, y_pred=y_rf_pred))

In [ ]:
svc =  SVC(random_state=0,verbose=1,probability=True)
svc.fit(x_train, y_train)
y_svc_pred = svc.predict(x_test)
y_svc_proba = svc.predict_proba(x_test)

print(cm(y_true=y_test, y_pred=y_svc_pred))
print(cr(y_true=y_test, y_pred=y_svc_pred))

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 50, 20), 
    verbose=1, 
    early_stopping=True, 
)

mlp.fit(x_train, x_test)
y_mlp_pred = mlp.predict(x_test)
y_mlp_proba = mlp.predict_proba(x_test)

print(cm(y_true=y_test, y_pred=y_mlp_pred))
print(cr(y_true=y_test, y_pred=y_mlp_pred))

In [ ]:
# XL 데이터의 예측 값
y_rf_proba_XL = rf.predict_proba(y_test_XL)
y_svc_proba_XL = svc.predict_proba(y_test_XL)
y_mlp_proba_XL = mlp.predict_proba(y_test_XL)

In [ ]:
np.savez_compressed("./npy/ML_prob.npz", rf_pred=y_rf_proba, svc_pred=y_svc_proba, mlp_pred=y_mlp_proba,
rf_pred_XL=y_rf_proba_XL, svc_pred_XL=y_svc_proba_XL, mlp_pred_XL=y_mlp_proba_XL,
label = y_test, label_XL =y_test_XL)